In [4]:
!pwd

/Users/aha/Desktop/Python/SF


In [7]:
!cd ../

In [20]:
!ls -al

total 776
drwxr-xr-x   9 aha  staff     306 Jan  8 11:40 .
drwxr-xr-x  21 aha  staff     714 Jan  8 11:26 ..
drwxr-xr-x   3 aha  staff     102 Jan  8 11:28 .ipynb_checkpoints
-rw-r--r--   1 aha  staff   50708 Jan  8 11:31 12081400(10511).ods
-rw-r--r--   1 aha  staff  239616 Jan  8 11:31 12081400(10511).xls
-rw-r--r--   1 aha  staff   77525 Jan  8 11:28 12081400(10511).zip
-rw-r--r--@  1 aha  staff   13044 Jan  8 11:40 HW.ipynb
-rw-r--r--   1 aha  staff    2915 Jan  8 11:39 export_LC.csv
-rw-r--r--   1 aha  staff    3486 Jan  8 11:33 import_LC.csv


## 處理流程

- 下載檔案 Extract
- 基本的檔案處理 ( 解壓縮, 轉換格式...等) Transformation
- 資料格式處理 T
- 資料輸出 Loading



In [1]:
import requests

## 下載

In [3]:
def download_file(folder_path,url):
    local_filename = folder_path+url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
                #f.flush() commented by recommendation from J.F.Sebastian
    return local_filename
download_file("./","http://www.banking.gov.tw/webdowndoc?file=/stat/lc/12081400(10511).zip")




'./12081400(10511).zip'

## 解壓縮

In [11]:
import zipfile
zf = zipfile.ZipFile("12081400(10511).zip",'r',zipfile.ZIP_DEFLATED)#打开已存在的zip文件
#zf.extractall("C:/Users/Administrator/Desktop/helloworld/Code/trade/")#解压到当前目录
zf.extractall("./")#解压到当前目录

檢查解壓縮好的資料

In [12]:
!ls -al

total 760
drwxr-xr-x   7 aha  staff     238 Jan  8 11:31 .
drwxr-xr-x  21 aha  staff     714 Jan  8 11:26 ..
drwxr-xr-x   3 aha  staff     102 Jan  8 11:28 .ipynb_checkpoints
-rw-r--r--   1 aha  staff   50708 Jan  8 11:31 12081400(10511).ods
-rw-r--r--   1 aha  staff  239616 Jan  8 11:31 12081400(10511).xls
-rw-r--r--   1 aha  staff   77525 Jan  8 11:28 12081400(10511).zip
-rw-r--r--@  1 aha  staff   14151 Jan  8 11:30 HW.ipynb


In [24]:
bank = {"銀行類別":"本國銀行","時間":"201611","業務項目":"進口開狀"}
bank

{'時間': '201611', '業務項目': '進口開狀', '銀行類別': '本國銀行'}

In [43]:
import xlrd
import sys
#book = xlrd.open_workbook("C:/Users/Administrator/Desktop/helloworld/Code/trade/12081400(10511).xls")
book = xlrd.open_workbook("./12081400(10511).xls")
sh = book.sheet_by_index(1)
banks = []
def extract_bank(row_s,d):
    bank = {"銀行類別":"本國銀行","時間":d,"業務項目":"進口開狀"}
    name_zh = sh.cell_value(rowx=row_s,colx = 0).replace("\u3000","").encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding)
    this_month = int(sh.cell_value(rowx=row_s,colx = 2).replace(",",""))*1000
    sum_month = int(sh.cell_value(rowx=row_s,colx = 5).replace(",",""))*1000
    bank["本月"] = this_month
    bank["本年累計"] = sum_month
    bank["銀行中文名稱"] = name_zh
    return bank

#for i in range(8,47):
#    banks.append(extract_bank(i))

In [28]:
read_Flag = 0
index = 0
while 1:
    ## 狀態控制
    if read_Flag == 0 and sh.cell_value(rowx=index,colx = 0).replace("\u3000","")=="總計":
        read_Flag = 1
    if read_Flag == 1 and sh.cell_value(rowx=index,colx = 0).replace("\u3000","") =="":
        break
    ## 資料處理
    if read_Flag == 1:
        banks.append(extract_bank(index))
    index = index + 1
    

### 寫出進口開狀

In [55]:
def writeToLC(date,banks):
    with open('./import_LC_'+date+'.csv','w+') as f:
        f.write("銀行中文名稱,本月,本年累計,銀行類別,時間,業務項目")
        for bank in banks:
            f.write("\n")
            f.write(bank["銀行中文名稱"]+",")
            f.write("%d,"%(bank["本月"]))
            f.write("%d,"%(bank["本年累計"]))
            f.write(bank["銀行類別"]+",")
            f.write(bank["時間"]+",")
            f.write(bank["業務項目"])
#writeToLC('201611')            

In [57]:
date = 10510
def month_job(date):
    download_file("./","http://www.banking.gov.tw/webdowndoc?file=/stat/lc/12081400(%d).zip"%date)
    zf = zipfile.ZipFile("12081400(%d).zip"%date,'r',zipfile.ZIP_DEFLATED)#打开已存在的zip文件
    #zf.extractall("C:/Users/Administrator/Desktop/helloworld/Code/trade/")#解压到当前目录
    zf.extractall("./")#解压到当前目录
    book = xlrd.open_workbook("./12081400(%d).xls"%date)
    sh = book.sheet_by_index(1)
    banks = []
    read_Flag = 0
    index = 0

    while 1:
        ## 狀態控制
        if read_Flag == 0 and sh.cell_value(rowx=index,colx = 0).replace("\u3000","")=="總計":
            read_Flag = 1
        if read_Flag == 1 and sh.cell_value(rowx=index,colx = 0).replace("\u3000","") =="":
            break
        ## 資料處理
        if read_Flag == 1:
            banks.append(extract_bank(index,'%d'%(date+191100)))
        index = index + 1
    writeToLC('%d'%(date+191100),banks)    


In [59]:
month_job(10508)

## 出口押匯

In [18]:
import xlrd
import sys
#book = xlrd.open_workbook("C:/Users/Administrator/Desktop/helloworld/Code/trade/12081400(10511).xls")
book = xlrd.open_workbook("./12081400(10511).xls")
sh = book.sheet_by_index(4)
banks = []
def extract_bank(row_s):
    bank = {"銀行類別":"本國銀行","時間":"201611","業務項目":"出口押匯"}
    name_zh = sh.cell_value(rowx=row_s,colx = 0).replace("\u3000","").encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding)
    this_month = int(sh.cell_value(rowx=row_s,colx = 2).replace(",",""))*1000
    sum_month = int(sh.cell_value(rowx=row_s,colx = 5).replace(",",""))*1000
    bank["本月"] = this_month
    bank["本年累計"] = sum_month
    bank["銀行中文名稱"] = name_zh
    return bank
for i in range(8,47):
    banks.append(extract_bank(i))

## 寫出出口開狀檔案

In [19]:
with open('./export_LC.csv','w+') as f:
    f.write("銀行中文名稱,本月,本年累計,銀行類別,時間,\n")
    for bank in banks:
        f.write(bank["銀行中文名稱"]+",")
        f.write("%d,"%(bank["本月"]))
        f.write("%d,"%(bank["本年累計"]))
        f.write(bank["銀行類別"]+",")
        f.write(bank["時間"]+",")
        f.write(bank["業務項目"]+",\n")
        

!dir/w

download_file('C:/Users/Administrator/Desktop/helloworld/Code/trade/',"http://www.banking.gov.tw/webdowndoc?file=/stat/lc/12081400(10511).zip")

import xlrd
book = xlrd.open_workbook("./12081400(10511)/12081400(10511).xls")
sh = book.sheet_by_index(1)

banks = []

bank = {"銀行類別":"本國銀行","時間":"201611","業務項目":"進口開狀"}

In [150]:
name_zh = sh.cell_value(rowx=8,colx = 0).replace("\u3000","")
name_zh
name_zh = sh.cell_value(rowx=28,colx = 0).replace("\u3000","")
name_zh.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding)
i

'?豐(台灣)商業銀行'

this_month = sh.cell_value(rowx=8,colx = 2)
this_month

sum_month = sh.cell_value(rowx = 8, colx = 5)
sum_month

print(book.nsheets)
print(sh.ncols)
print(sh.nrows)

bank["本月"] = this_month
bank["本年累計"] = sum_month
bank["銀行中文名稱"] = name_zh
bank

banks.append(bank)
banks


banks

import sys
print(name_zh.encode(sys.stdin.encoding, "replace").decode(sys.stdin.encoding))

for i in range(7,48):
    this_month = (int(sh.cell_value(rowx=i,colx = 2).replace(",",""))*1e3)
    print(this_month)

import xlrd
workbook = xlrd.open_workbook("./12081400(10511)/12081400(10511).xls")
sh = workbook.sheet_by_index(1)
print(workbook.nsheets)
print(workbook.sheet_names())
print(sh.col_values(0))

print(sh.ncols)

for i in range(7,48):
    thismonth = int(sh.cell_value(rowx=i,colx = 2).replace(",",""))
    print(this_month)

thismonth = sh.cell_value(rowx=8,colx = 2).replace(",","")
thismonth = int(thismonth)*1000
type(thismonth)
print(thismonth)


# coding=utf-8
import zipfile
zf = zipfile.ZipFile("./test.zip",'r',zipfile.ZIP_DEFLATED)#打开已存在的zip文件
zf.extractall("./")#解压到当前目录
#zf.extractall('..') #解压到上一级目录
#zf.extractall('C:/')#解压到C:/tmp文件夹，文件夹不存在则创建
